<a href="https://colab.research.google.com/github/LAVANYA260506/Image-classification-/blob/main/image_classification_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from PIL import Image

from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dataset


In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [ ]:
train_data=torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    transform=transform,
    download=True
)

test_data=torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    transform=transform,
    download=True
)

In [ ]:
train_loader= DataLoader(
    train_data,
    batch_size=32,
    shuffle=True,
    num_workers=2
)

test_loader= DataLoader(
    test_data,
    batch_size=32,
    shuffle=True,
    num_workers=2
)

In [ ]:
image,label = train_data[0]

In [ ]:
image.size()

In [ ]:
class_names = ['plane','car','bird','cat','deer','dog','frog','truck','horse','ship']

In [ ]:
from typing import Self
class CnnModel(nn.Module):
  def __init__(self):

    super().__init__()
    self.conv1 = nn.Conv2d(3,12,5)
    self.pool = nn.MaxPool2d(2,2)
    self.conv2 = nn.Conv2d(12,24,5)
    self.fc1 = nn.Linear(24*5*5,120)
    self.fc2 = nn.Linear(120,84)
    self.fc3 = nn.Linear(84,10)

  def forward(self,x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = torch.flatten(x,1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [ ]:
model = CnnModel()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=0.001,momentum=0.9)

In [ ]:
for epoch in range(30):
    print(f"Training epoch {epoch}..")

    running_loss = 0.0

    for i , data in enumerate(train_loader):
            inputs , labels = data

            optimizer.zero_grad()

            outputs = model(inputs)

            loss = loss_function(outputs,labels)
            loss.backward()

            optimizer.step()

            running_loss += loss.item()

    print(f'loss  : {running_loss/len(train_loader) : 4f}')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
torch.save(model.state_dict(),'trained_model.pth')

In [ ]:
model = CnnModel()
model.load_state_dict(torch.load('trained_model.pth'))

In [ ]:
correct =0
total =0

model.eval()

with torch.no_grad():
  for data in test_loader:
    images , labels = data
    outputs = model(images)
    _,predicted = torch.max(outputs , 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'accuracy : {accuracy}%')

In [ ]:
new_transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

def load_image(image_path):
  image = Image.open(image_path)
  image = new_transform(image)
  image= image.unsqueeze(0)
  return image

image_paths = ['/content/doggie.jpeg','/content/froggie.jpeg']
images = [load_image(img) for img in image_paths]

model.eval()

with torch.no_grad():
  for image in images:
    output =model(image)
    _,predicted = torch.max(output,1)
    print(f'prediction : {class_names[predicted.item()]}')
